In [37]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(200_000_000_000)

import intake
import geopandas as gpd
import pandas as pd
from calitp_data_analysis.gcs_geopandas import GCSGeoPandas
from calitp_data_analysis import geography_utils

from shared_utils import webmap_utils, gtfs_utils_v2, rt_dates
import branca

In [22]:
from update_vars import analysis_date, GCS_FILE_PATH

In [26]:
gcsgp = GCSGeoPandas()

In [23]:
analysis_date

'2025-11-05'

In [3]:
catalog = intake.open_catalog('catalog.yml')

## Rail/Ferry/BRT Major Stops

In [4]:
ca = catalog.ca_boundary.read().to_crs(geography_utils.CA_NAD83Albers_m)

In [5]:
rail_ferry_brt = catalog.rail_brt_ferry_stops.read()

In [6]:
rail_ferry_brt = rail_ferry_brt.clip(ca)

In [7]:
hqta_types = rail_ferry_brt.hqta_type.unique()

In [8]:
cmap = branca.colormap.step.Accent_08

In [9]:
color_dict = webmap_utils.categorical_cmap(cmap, hqta_types)

In [10]:
color_dict

{'major_stop_rail': (127, 201, 127),
 'major_stop_brt': (190, 174, 212),
 'major_stop_ferry': (253, 192, 134)}

In [11]:
rail_ferry_brt = rail_ferry_brt.assign(color = rail_ferry_brt.hqta_type.map(color_dict))

In [12]:
webmap_utils.set_state_export(rail_ferry_brt, filename = 'rail_ferry_brt3', cache_seconds=0)

/home/jovyan/data-analyses/_shared_utils/shared_utils/webmap_utils.py:126: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = (gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean())


file already exists at calitp-map-tiles/testing/rail_ferry_brt3.geojson.gz, not overwriting, use overwrite = True to overwrite


{'state_dict': {'name': 'null',
  'layers': [{'name': 'Map',
    'url': 'https://storage.googleapis.com/calitp-map-tiles/testing/rail_ferry_brt3.geojson.gz',
    'properties': {'stroked': False, 'highlight_saturation_multiplier': 0.5}}],
  'lat_lon': (36.787838477122335, -121.16992878911985),
  'zoom': 13},
 'spa_link': 'https://embeddable-maps.calitp.org/?state=eyJuYW1lIjogIm51bGwiLCAibGF5ZXJzIjogW3sibmFtZSI6ICJNYXAiLCAidXJsIjogImh0dHBzOi8vc3RvcmFnZS5nb29nbGVhcGlzLmNvbS9jYWxpdHAtbWFwLXRpbGVzL3Rlc3RpbmcvcmFpbF9mZXJyeV9icnQzLmdlb2pzb24uZ3oiLCAicHJvcGVydGllcyI6IHsic3Ryb2tlZCI6IGZhbHNlLCAiaGlnaGxpZ2h0X3NhdHVyYXRpb25fbXVsdGlwbGllciI6IDAuNX19XSwgImxhdF9sb24iOiBbMzYuNzg3ODM4NDc3MTIyMzM1LCAtMTIxLjE2OTkyODc4OTExOTg1XSwgInpvb20iOiAxM30='}

In [14]:
rail_ferry_brt

,schedule_gtfs_dataset_key_primary,stop_id,stop_name,route_id,route_type,hqta_type,geometry,color
778,1165b1474df778cb0fc3ba9246e32035,SNS,Salinas,36924,2,major_stop_rail,POINT (-147851.418 -147353.146),"(127, 201, 127)"
1055,118988ea737b00fd3b3129187c8751bc,70322,Gilroy Caltrain Station Southbound,South County,2,major_stop_rail,POINT (-139178.464 -111445.406),"(127, 201, 127)"
1054,118988ea737b00fd3b3129187c8751bc,70321,Gilroy Caltrain Station Northbound,South County,2,major_stop_rail,POINT (-139166.194 -111439.717),"(127, 201, 127)"
1052,118988ea737b00fd3b3129187c8751bc,70311,San Martin Caltrain Station Northbound,South County,2,major_stop_rail,POINT (-142918.111 -102295.656),"(127, 201, 127)"
1053,118988ea737b00fd3b3129187c8751bc,70312,San Martin Caltrain Station Southbound,South County,2,major_stop_rail,POINT (-142994.142 -102135.611),"(127, 201, 127)"
...,...,...,...,...,...,...,...,...
1138,1451f537bdcefd0e8ba827d12c4ef4b8,13858,California St & Divisadero St,1,3,major_stop_brt,POINT (-214642.193 -22613.835),"(190, 174, 212)"
1141,1451f537bdcefd0e8ba827d12c4ef4b8,13885,California St & Pierce St,1,3,major_stop_brt,POINT (-214284.021 -22582.128),"(190, 174, 212)"
1140,1451f537bdcefd0e8ba827d12c4ef4b8,13884,California St & Pierce St,1,3,major_stop_brt,POINT (-214343.273 -22573.598),"(190, 174, 212)"
2764,ff345d53ce8a0e1faf13973402a1dc22,43007,Tiburon (No Service to Angel Island),TRIA,4,major_stop_ferry,POINT (-215721.361 -13173.100),"(253, 192, 134)"


## Analysis Segments and Key Stops

In [20]:
hqta_segments = catalog.hqta_segments.read()

In [21]:
hqta_segments.head(3)

,schedule_gtfs_dataset_key,route_id,route_key,segment_sequence,geometry,hqta_segment_id,route_direction,fwd_azimuth,fwd_azimuth_360
0,eca94265831d5c499800dd921ecf4011,16672,001103e6b305855393a09d8958648e2d,0,"LINESTRING (-91399.426 134141.367, -91391.455 ...",3312151719,east-west,54.298625,54.298625
1,eca94265831d5c499800dd921ecf4011,16672,001103e6b305855393a09d8958648e2d,1,"LINESTRING (-90554.269 134735.738, -90536.160 ...",2993445937,north-south,43.196810,43.196810
2,eca94265831d5c499800dd921ecf4011,16672,001103e6b305855393a09d8958648e2d,2,"LINESTRING (-89690.181 135637.680, -89627.870 ...",728042891,east-west,68.511655,68.511655


In [29]:
path = f'{GCS_FILE_PATH}all_bus.parquet'

In [30]:
path

'gs://calitp-analytics-data/data-analyses/high_quality_transit_areas/all_bus.parquet'

In [40]:
max_arrivals_by_stop = pd.read_parquet(f"{GCS_FILE_PATH}max_arrivals_by_stop.parquet")

In [41]:
max_arrivals_by_stop.head(10)

,schedule_gtfs_dataset_key,stop_id,am_max_trips,route_dir,pm_max_trips,am_max_trips_hr,pm_max_trips_hr,n_trips,route_dir_count,route_id
0,0089bd1b0a2b78a8590d8749737d7146,40414,12,35_1,14.0,4.00,3.5,26.0,1,35
1,0089bd1b0a2b78a8590d8749737d7146,40467,11,35_1,14.0,3.67,3.5,25.0,1,35
2,0089bd1b0a2b78a8590d8749737d7146,44005,13,35_1,14.0,4.33,3.5,27.0,1,35
3,11865a0a240039d4479ded2595c832a5,1008,9,2_0,12.0,3.00,3.0,21.0,1,2
4,11865a0a240039d4479ded2595c832a5,1214,9,2_0,12.0,3.00,3.0,21.0,1,2
5,11865a0a240039d4479ded2595c832a5,1245,9,1_0,12.0,3.00,3.0,21.0,1,1
6,11865a0a240039d4479ded2595c832a5,1251,9,1_0,12.0,3.00,3.0,21.0,1,1
7,11865a0a240039d4479ded2595c832a5,1260,9,1_0,12.0,3.00,3.0,21.0,1,1
8,11865a0a240039d4479ded2595c832a5,1262,9,1_0,12.0,3.00,3.0,21.0,1,1
9,11865a0a240039d4479ded2595c832a5,1264,9,1_0,12.0,3.00,3.0,21.0,1,1


In [84]:
gdf = gcsgp.read_parquet(path)

In [85]:
gdf.route_dir_count.unique()

array([1, 2, 3, 4])

In [86]:
stops = gcsgp.read_parquet(f"{GCS_FILE_PATH}stops_with_lookback.parquet")

In [87]:
stops.columns

Index(['feed_key', 'stop_id', 'stop_name', 'geometry', 'analysis_date',
       'schedule_gtfs_dataset_key', 'analysis_name'],
      dtype='object')

In [88]:
stops = stops[['stop_id', 'stop_name', 'analysis_date',
      'schedule_gtfs_dataset_key', 'analysis_name', 'geometry']]

In [89]:
stops = stops.rename(columns={'geometry': 'stop_geometry'})

In [90]:
gdf = gdf.merge(stops, on = ['stop_id', 'schedule_gtfs_dataset_key'])

In [91]:
map1 = gdf.copy()[['route_id', 'stop_id', 'geometry', 'route_direction',
   'fwd_azimuth_360', 'circuitous_segment', 'hq_transit_corr',
   'ms_precursor', 'analysis_name']]

In [56]:
cmap

In [95]:
cmap = branca.colormap.step.Set1_03

In [96]:
cmap_dict = webmap_utils.categorical_cmap(cmap, ['segment', 'stop'])
cmap_dict

{'stop': (228, 26, 28), 'segment': (55, 126, 184)}

In [97]:
map1['color'] = [cmap_dict['segment']] * len(map1)

In [99]:
state = webmap_utils.set_state_export(map1, filename = 'hqta_segments', cache_seconds=0,
                                     map_title='HQTA Segments', overwrite=True)

/home/jovyan/data-analyses/_shared_utils/shared_utils/webmap_utils.py:126: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = (gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean())


In [100]:
state

{'state_dict': {'name': 'null',
  'layers': [{'name': 'HQTA Segments',
    'url': 'https://storage.googleapis.com/calitp-map-tiles/testing/hqta_segments.geojson.gz',
    'properties': {'stroked': False, 'highlight_saturation_multiplier': 0.5}}],
  'lat_lon': (35.22778023895391, -119.46497161196228),
  'zoom': 13},
 'spa_link': 'https://embeddable-maps.calitp.org/?state=eyJuYW1lIjogIm51bGwiLCAibGF5ZXJzIjogW3sibmFtZSI6ICJIUVRBIFNlZ21lbnRzIiwgInVybCI6ICJodHRwczovL3N0b3JhZ2UuZ29vZ2xlYXBpcy5jb20vY2FsaXRwLW1hcC10aWxlcy90ZXN0aW5nL2hxdGFfc2VnbWVudHMuZ2VvanNvbi5neiIsICJwcm9wZXJ0aWVzIjogeyJzdHJva2VkIjogZmFsc2UsICJoaWdobGlnaHRfc2F0dXJhdGlvbl9tdWx0aXBsaWVyIjogMC41fX1dLCAibGF0X2xvbiI6IFszNS4yMjc3ODAyMzg5NTM5MSwgLTExOS40NjQ5NzE2MTE5NjIyOF0sICJ6b29tIjogMTN9'}

In [101]:
map2 = gdf.copy()[['stop_id', 'stop_name', 'am_max_trips_hr',
                  'pm_max_trips_hr', 'route_dir_count', 'analysis_name',
                  'stop_geometry']].set_geometry('stop_geometry')

In [102]:
map2['color'] = [cmap_dict['stop']] * len(map1)

In [103]:
webmap_utils.set_state_export(map2, filename = 'hqta_segment_key_stops', cache_seconds=0,
                             existing_state=state['state_dict'], map_title='Stops and Segments', overwrite=True)

/home/jovyan/data-analyses/_shared_utils/shared_utils/webmap_utils.py:126: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = (gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean())


{'state_dict': {'name': 'null',
  'layers': [{'name': 'HQTA Segments',
    'url': 'https://storage.googleapis.com/calitp-map-tiles/testing/hqta_segments.geojson.gz',
    'properties': {'stroked': False, 'highlight_saturation_multiplier': 0.5}},
   {'name': 'Stops and Segments',
    'url': 'https://storage.googleapis.com/calitp-map-tiles/testing/hqta_segment_key_stops.geojson.gz',
    'properties': {'stroked': False, 'highlight_saturation_multiplier': 0.5}}],
  'lat_lon': (35.228084485138616, -119.46550119462731),
  'zoom': 13},
 'spa_link': 'https://embeddable-maps.calitp.org/?state=eyJuYW1lIjogIm51bGwiLCAibGF5ZXJzIjogW3sibmFtZSI6ICJIUVRBIFNlZ21lbnRzIiwgInVybCI6ICJodHRwczovL3N0b3JhZ2UuZ29vZ2xlYXBpcy5jb20vY2FsaXRwLW1hcC10aWxlcy90ZXN0aW5nL2hxdGFfc2VnbWVudHMuZ2VvanNvbi5neiIsICJwcm9wZXJ0aWVzIjogeyJzdHJva2VkIjogZmFsc2UsICJoaWdobGlnaHRfc2F0dXJhdGlvbl9tdWx0aXBsaWVyIjogMC41fX0sIHsibmFtZSI6ICJTdG9wcyBhbmQgU2VnbWVudHMiLCAidXJsIjogImh0dHBzOi8vc3RvcmFnZS5nb29nbGVhcGlzLmNvbS9jYWxpdHAtbWFwLXRpbGVzL3